## API Test for ChatBot v1 (Pre-Production)

### Set ENV Variables using the below command/script

In [ ]:
# S3_ENDPOINT = os.getenv('S3_ENDPOINT')
# S3_ACCESS_KEY = os.getenv('S3_ACCESS_KEY')
# S3_SECRET_KEY = os.getenv('S3_SECRET_KEY')
# S3_BUCKET = os.getenv('S3_BUCKET', 'chatbotv1')
# S3_BASE_FOLDER = os.getenv('S3_BASE_FOLDER', 'org')
export API_KEY="Kj_h8vXY5xZP3PmYqC7RLL9K3vhvU_hBZxXF4T9dr_A" \
export S3_ENDPOINT="https://objectstore.e2enetworks.net" \
export S3_ACCESS_KEY="P3JIUOIQ3BLXOEGUY0H9" \
export S3_SECRET_KEY="1EVTV1111SFIIGTPGHXDJ63R390O2MRRVX6ZLTA3" \
export NVIDIA_API_KEY="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"

In [1]:
import ipywidgets as widgets
from IPython.display import display, HTML
import requests
import json
from typing import Dict, Any
import time
import uuid

class ChatbotDemo:
    def __init__(self, base_url: str = "http://164.52.207.75:9000", api_key: str = "Kj_h8vXY5xZP3PmYqC7RLL9K3vhvU_hBZxXF4T9dr_A"):
        self.base_url = base_url.rstrip('/')
        self.api_key = api_key
        self.headers = {
            'Content-Type': 'application/json',
            'x-api-key': self.api_key
        }
        
        # Create widgets for organization ID and session ID (username)
        self.org_input = widgets.Text(
            placeholder='Enter Organization ID (e.g., 00001)',
            description='Organization ID:',
            value='00001',
            layout=widgets.Layout(width='300px')
        )
        
        self.session_input = widgets.Text(
            placeholder='Enter your name',
            description='Your Name:',
            value='',
            layout=widgets.Layout(width='300px')
        )
        
        # Session ID help text
        self.session_help = widgets.HTML(
            value='<span style="color: #666; font-size: 0.8em; margin-left: 10px;">This will be used as your session ID for conversation continuity</span>'
        )
        
        # Create a unique default session ID
        self.default_session_id = f"user-{uuid.uuid4().hex[:8]}"
        
        # Create widgets for query input and buttons
        self.query_input = widgets.Text(
            placeholder='Type your question here...',
            description='Query:',
            layout=widgets.Layout(width='70%')
        )
        
        self.send_button = widgets.Button(
            description='Send',
            button_style='primary',
            layout=widgets.Layout(width='100px')
        )
        
        self.clear_button = widgets.Button(
            description='Clear Chat',
            button_style='warning',
            layout=widgets.Layout(width='120px')
        )
        
        self.output_area = widgets.Output(
            layout=widgets.Layout(
                width='80%',
                height='400px',
                border='1px solid #ddd',
                padding='10px',
                margin='10px 0',
                overflow_y='auto'
            )
        )
        
        # Style for response types
        self.source_styles = {
            'corpus': '<span style="color: #2E86C1;">📚 Response from the ERP Corpus</span>',
            'llm': '<span style="color: #27AE60;">🤖 Response from LLM</span>'
        }
        
        # Set up the layout
        self.setup_layout()
    
    def setup_layout(self):
        """Set up the widget layout"""
        # Create org and session input containers
        org_container = widgets.HBox([
            self.org_input
        ], layout=widgets.Layout(
            margin='10px 0',
            display='flex',
            align_items='center'
        ))
        
        session_container = widgets.HBox([
            self.session_input,
            self.session_help
        ], layout=widgets.Layout(
            margin='10px 0',
            display='flex',
            align_items='center'
        ))
        
        # Create input container
        input_container = widgets.HBox([
            self.query_input,
            self.send_button
        ], layout=widgets.Layout(
            margin='10px 0',
            display='flex',
            align_items='center'
        ))
        
        # Create button container for additional buttons
        button_container = widgets.HBox([
            self.clear_button
        ], layout=widgets.Layout(
            margin='10px 0',
            display='flex',
            align_items='center',
            justify_content='flex-end'
        ))
        
        # Create main container
        self.main_container = widgets.VBox([
            widgets.HTML(
                value='<h2 style="color: #444;">SMT ChatBot v1.8 Demo</h2>'
            ),
            org_container,
            session_container,
            input_container,
            button_container,
            self.output_area
        ], layout=widgets.Layout(
            padding='20px',
            width='100%'
        ))
        
        # Add button click handlers
        self.send_button.on_click(self.handle_send)
        self.clear_button.on_click(self.handle_clear)
        
        # Add enter key handler
        self.query_input.on_submit(self.handle_send)
        
    def get_session_id(self) -> str:
        """Get the session ID from the input or use the default"""
        user_name = self.session_input.value.strip()
        if user_name:
            # Use the user's name as the session ID
            return user_name
        else:
            # Use the default session ID
            return self.default_session_id
        
    def send_query(self, query: str, organization_id: str) -> Dict[str, Any]:
        """Send query to API with organization ID and session ID"""
        try:
            # Get session ID (user name)
            session_id = self.get_session_id()
            
            payload = {
                "query": query,
                "organization_id": organization_id,
                "session_id": session_id,
                "user_id": "demo-user"
            }
            
            start_time = time.time()
            response = requests.post(
                f"{self.base_url}/api/chat",
                headers=self.headers,
                json=payload
            )
            end_time = time.time()
            
            response_data = response.json()
            # Add response time for display
            response_data['response_time'] = (end_time - start_time) * 1000  # in ms
            
            return response_data
            
        except Exception as e:
            return {
                'status': 'error',
                'error': str(e)
            }
    
    def handle_send(self, button):
        """Handle send button click or enter key"""
        query = self.query_input.value
        organization_id = self.org_input.value.strip()
        session_id = self.get_session_id()
        
        if not query.strip():
            return
        
        if not organization_id:
            with self.output_area:
                display(HTML(
                    '<div style="color: #E74C3C; margin-top: 10px;">'
                    '❌ Error: Organization ID is required'
                    '</div>'
                ))
            return
            
        # Clear input
        self.query_input.value = ''
        
        with self.output_area:
            # Show organization, session and user query
            display(HTML(
                f'<div style="margin-bottom: 10px;">'
                f'<span style="color: #666;">👤 You (Org: {organization_id}, Name: {session_id}):</span><br>'
                f'<span style="margin-left: 20px;">{query}</span>'
                f'</div>'
            ))
            
            # Show "typing" indicator
            display(HTML(
                '<div id="typing" style="color: #666; margin: 10px 0;">'
                '⌛ Processing...'
                '</div>'
            ))
            
            # Get response
            result = self.send_query(query, organization_id)
            
            # Remove typing indicator
            display(HTML('<script>document.getElementById("typing").remove()</script>'))
            
            if result.get('status') == 'success':
                # Get source indicator
                source = result.get('source', 'system')
                source_indicator = self.source_styles.get(
                    source,
                    '<span style="color: #666;">🔧 System Response</span>'
                )
                
                # Get organization ID from response
                resp_org_id = result.get('organization_id', organization_id)
                
                # Format the response
                response_text = result.get("response", "").replace('\n', '<br>')
                
                # Show response with organization ID
                display(HTML(
                    f'<div style="margin-top: 10px;">'
                    f'{source_indicator} (Organization {resp_org_id})<br>'
                    f'<span style="margin-left: 20px;">{response_text}</span>'
                    f'</div>'
                ))
                
                # Show confidence if available
                if 'confidence' in result and result['confidence']:
                    confidence = result['confidence']
                    display(HTML(
                        f'<div style="color: #777; font-size: 0.8em; margin-top: 5px; margin-left: 20px;">'
                        f'Confidence: Main={confidence.get("main", "N/A"):.2f}, Sub={confidence.get("sub", "N/A"):.2f}'
                        f'</div>'
                    ))
                
                # Show response time
                if 'response_time' in result:
                    display(HTML(
                        f'<div style="color: #777; font-size: 0.8em; margin-top: 5px; margin-left: 20px;">'
                        f'Response time: {result["response_time"]:.2f} ms'
                        f'</div>'
                    ))
            else:
                # Show error
                error_msg = result.get('detail', 'Failed to get response. Please try again.')
                display(HTML(
                    f'<div style="color: #E74C3C; margin-top: 10px;">'
                    f'❌ Error: {error_msg}'
                    f'</div>'
                ))
    
    def handle_clear(self, button):
        """Clear the output area"""
        self.output_area.clear_output()
        
        with self.output_area:
            display(HTML(
                '<div style="color: #666; text-align: center;">'
                'Chat cleared. Start a new conversation!'
                '</div>'
            ))
    
    def display(self):
        """Display the demo interface"""
        display(self.main_container)

# Create and display the demo
demo = ChatbotDemo(api_key="Kj_h8vXY5xZP3PmYqC7RLL9K3vhvU_hBZxXF4T9dr_A")
demo.display()

/tmp/ipykernel_84558/3211908171.py:140: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  self.query_input.on_submit(self.handle_send)
